In [1]:
import pandas as pd
import numpy as np
import sys
import os

# Add project root to path
sys.path.append(r"D:\data-problem-solving")

from db import load_table, engine

df =  load_table('ride_requests')

In [2]:
df['request_date'] = pd.to_datetime(df['request_date'])

monthly = df.groupby(df['request_date'].dt.to_period('M')).agg(
    total_distance = ('distance_to_travel', 'sum'),
    total_cost = ('monetary_cost', 'sum')
).reset_index()

monthly['actual_distance_per_dollar'] = monthly['total_distance']/monthly['total_cost']

monthly['forecasted_distance_per_dollar'] = monthly['actual_distance_per_dollar'].shift(1)

monthly['error'] = monthly['actual_distance_per_dollar'] - monthly['forecasted_distance_per_dollar']
rmse = np.sqrt((monthly['error'].dropna() ** 2).mean())
rmse = round(rmse,2)

monthly_rmse = monthly.copy()
monthly_rmse.loc[len(monthly_rmse)] = ['RMSE', np.nan, np.nan, np.nan, np.nan, rmse]

monthly_rmse['request_date'] = monthly_rmse['request_date'].astype(str)

print(monthly_rmse)

  request_date  total_distance  total_cost  actual_distance_per_dollar  \
0      2020-01          163.95       29.24                    5.607045   
1      2020-02          112.82       19.43                    5.806485   
2      2020-03           25.04        7.19                    3.482615   
3         RMSE             NaN         NaN                         NaN   

   forecasted_distance_per_dollar    error  
0                             NaN      NaN  
1                        5.607045  0.19944  
2                        5.806485 -2.32387  
3                             NaN  1.65000  
